In [1]:
''' initialise development environment '''

# set auto reload imported modules tagged
%load_ext autoreload
%autoreload 2


In [2]:
''' import optics package '''

# add custom python packages directory to path
import sys
sys.path.append('/home/brendan/dev/optics')


%matplotlib widget


# import path tracing and image transformation engine
import optics


In [3]:
''' Imports '''

# nd array manipulation
import numpy as np

# image manipulation
from scipy import ndimage

# plotting with matplotlib, interactive notebook, 3d toolkit

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


In [5]:
''' Generate Target Image '''

# set edge length; ensure odd
edge_len = 151

# generate pattern image (target)
test_image = optics.gen_image(edge_len)

# import image, scaled to edge length
#test_image = optics.import_image('../data/test-img-1.png', edge_len)


# initialise figure and axes, clean format
_w = 4; _h = 4
fig = plt.figure(figsize = (_w, _h))
#fig.canvas.layout.width = '{}in'.format(_w)
#fig.canvas.layout.height= '{}in'.format(_h)
ax = fig.add_subplot(111)
ax.grid([]); ax.set_xticks([]); ax.set_yticks([])


_img = test_image
#_img = ndimage.gaussian_filter(test_image, sigma = 1.)

ax.imshow(_img, cmap = 'bone_r', vmin = 0, vmax = 1)


plt.tight_layout()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
zero.shape

(151, 151)

In [20]:
zero.mean()

0.0

In [6]:


# set edge length; ensure odd
edge_len = 151

''' generate img '''

# generate pattern image (target)
#test_image = optics.gen_image(edge_len, style = 'grid')



# generate target range
div = [ (2.0 / (edge_len - 1)) for _ in range(2) ]
rng = [ [-1., 1.] for _ in range(2) ]

# prepare blank grid
grid = [ np.arange(rng[i][0], rng[i][1] + div[i], div[i]) for i in range(2) ]
zero = np.array([ [ 0. for xi in grid[0] ] for yi in grid[1] ])

# circles
if False:
    # set target origin
    orig = np.array([0., 0.])

    # calculate distance map
    dist = np.array([ [ np.sqrt(sum((np.array([xi, yi])-orig)**2)) for xi in grid[0] ] for yi in grid[1] ])


    # generate pattern from distance map
    patt = (np.cos(dist * 8 * 2 * np.pi) + 1.) / 2.0

    
# set grid line spacing
k = 15
# set grid line thickness
t = 1

# generate full grid
for i in range(2*k, edge_len - 2*k):
    for j in range(2*k, edge_len - 2*k):
        if (i % k <= t) or(i % k >= k-t) or (j % k <= t) or (j % k >= k-t):
            zero[i,j] = 1

patt = zero

# smooth edges for low res
patt = ndimage.gaussian_filter(patt, sigma = 0.5)



test_image = patt

# import image, scaled to edge length
#test_image = optics.import_image('../data/test-img-1.png', edge_len)

# initialise figure and axes, clean format
_w = 6; _h = 6
fig = plt.figure(figsize = (_w, _h))
#fig.canvas.layout.width = '{}in'.format(_w)
#fig.canvas.layout.height= '{}in'.format(_h)
ax = fig.add_subplot(111)
ax.grid([]); ax.set_xticks([]); ax.set_yticks([])


_img = test_image
#_img = ndimage.gaussian_filter(test_image, sigma = 1.)

ax.imshow(_img, cmap = 'bone_r', vmin = 0, vmax = 1)


plt.tight_layout()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
''' Generate Target Image '''

# set edge length; ensure odd
edge_len = 251

# generate pattern image (target)
test_image = optics.gen_image(edge_len, style = 'grid')

# import image, scaled to edge length
#test_image = optics.import_image('../data/test-img-1.png', edge_len)


# initialise figure and axes, clean format
_w = 4; _h = 4
fig = plt.figure(figsize = (_w, _h))
#fig.canvas.layout.width = '{}in'.format(_w)
#fig.canvas.layout.height= '{}in'.format(_h)
ax = fig.add_subplot(111)
ax.grid([]); ax.set_xticks([]); ax.set_yticks([])


_img = test_image
#_img = ndimage.gaussian_filter(test_image, sigma = 1.)

ax.imshow(_img, cmap = 'bone_r', vmin = 0, vmax = 1)


plt.tight_layout()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
''' Generate Initial Ray '''

# set height of target image (mm), and supersample factor
height = 5.
ss = 2.

# generate rays for image translation (list np.array[px, py, pz, vx, vy, vz] )
rays = optics.gen_img_rays(edge_len, height, test_image, ss)


501
251001
60272


In [9]:
''' Define Standard Optics '''

# get standard optical parameters
opt_params = optics.std_opt_params()

# overwrite standard optical parameters
opt_params = { **opt_params,
    
    #'eye_front': 300.,
    
    #'cornea_sph': 1.,
    #'cornea_axis': 0.,
    
    #'cornea_pow': np.sqrt(0.5),
    
    #'iris_dia': 4.,
    
    #'focus': 1.,
    
    #'lens_pow': np.sqrt(4.5),
    
    #'retina_thick': 17.2,
}

# generate standard optics chain
opts = optics.gen_optics(opt_params)


In [10]:
''' Calculate Ray Paths through Optics '''

# calculate ray paths through optics chain to retina
paths = optics.get_paths(rays, opts)


In [11]:
''' Generate Reverse Rays '''

# generate reverse rays for back propagation through reverse optics chain
back_rays = optics.gen_rev_rays(paths, opt_params)


In [12]:
''' Define Reverse Optics (Stigmatism) '''

# define stigmatism optics chain by optical parameters
rev_opt_params = {**opt_params,
    #'cornea_sph': opt_params['cornea_sph'] - 0.02,
    #'cornea_axis': opt_params['cornea_axis'] + 45.,
    'cornea_pow': opt_params['cornea_pow'] + 4 * np.sqrt(0.5)/100,
    #'cornea_f_rad': opt_params['cornea_f_rad'] - 0.2,
                  
}

# generate standard optics chain, overwrite existing params
rev_opts = optics.gen_optics_rev(rev_opt_params)


In [13]:
''' get ray paths through optics chain ''' 

# calculate reverse ray paths from retina, set initial refractive index
rev_paths = optics.get_paths(back_rays, rev_opts, n0 = 1.337)


In [14]:
''' Resample Translated Rays as Image'''

# build image by resample return rays over area
grid = optics.translate_image(test_image, ss, paths, rev_paths, height, edge_len)


In [15]:
# initialise figure and axes, clean format
_w = 7; _h = 4
fig = plt.figure(figsize = (_w, _h))
#fig.canvas.layout.width = '{}in'.format(_w)
#fig.canvas.layout.height= '{}in'.format(_h)

plt.subplot(121)
plt.grid(''); plt.xticks([]); plt.yticks([])

plt.imshow(test_image, cmap = 'bone_r', vmin = 0., vmax = 1.)


plt.subplot(122)
plt.grid(''); plt.xticks([]); plt.yticks([])

#_img = ndimage.filters.median_filter(grid, size = 4)
_img = grid.copy()

plt.imshow(_img, cmap = 'bone_r', vmin = 0., vmax = 1.)

#plt.savefig('./images/output-0s95.png', dpi = 200)

plt.tight_layout()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
# initialise figure and axes, clean format
_w = 7; _h = 4
fig = plt.figure(figsize = (_w, _h))
#fig.canvas.layout.width = '{}in'.format(_w)
#fig.canvas.layout.height= '{}in'.format(_h)

plt.subplot(121)
plt.grid(''); plt.xticks([]); plt.yticks([])

plt.imshow(test_image, cmap = 'bone_r', vmin = 0., vmax = 1.)


plt.subplot(122)
plt.grid(''); plt.xticks([]); plt.yticks([])

#_img = ndimage.filters.median_filter(grid, size = 4)
_img = grid.copy()

plt.imshow(_img, cmap = 'bone_r', vmin = 0., vmax = 1.)

#plt.savefig('./images/output-0s95.png', dpi = 200)

plt.tight_layout()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
''' initialise standard optics chain '''

# set edge length; ensure odd
edge_len = 151

# set height of target image (mm), and supersample factor
height = 4.
ss = 3.

# overwrite standard optical parameters
opt_params = {

    #'eye_front': 300.,

    #'cornea_sph': 1.,
    #'cornea_axis': 0.,

    #'cornea_pow': np.sqrt(0.5),

    #'iris_dia': 4.,

    #'focus': 1.,

    #'lens_pow': np.sqrt(4.5),

    #'retina_thick': 17.2,
}

# initialise standard optics chain
test_image, rays, opt_params, opts, paths, back_rays = optics.init_optics(edge_len, height, ss, opt_params)


22801
451
203401
20892


In [26]:
np.sqrt(0.5)/100

0.007071067811865475

In [27]:
''' define stigmatism and batch image translate '''

# define stigmatism, each parameter range as delta (min, max, step)
stig = {
    #'param': 'cornea_sph',
    #'range': [-.009, .010, .001]
    
    'param': 'cornea_pow',
    'range': [-.016, .017, .002],
    
}

# perform batch image translation over stigmatism parameter range, return image set
images = optics.batch_translate(test_image, edge_len, height, ss, opt_params, paths, stig, back_rays)


17
image 1/17
image 2/17
image 3/17
image 4/17
image 5/17
image 6/17
image 7/17
image 8/17
image 9/17
image 10/17
image 11/17
image 12/17
image 13/17
image 14/17
image 15/17
image 16/17
image 17/17


In [28]:
import pickle


In [29]:
with open('../data/subj-refr-foc-images', 'wb') as file:
    pickle.dump(images, file)
    

In [30]:
with open('../data/subj-refr-foc-images', 'rb') as file:
    images = pickle.load(file)


In [34]:

# initialise figure and axes, clean format
_w = 4; _h = 4
fig = plt.figure(figsize = (_w, _h))
fig.canvas.layout.width = '{}in'.format(_w)
fig.canvas.layout.height= '{}in'.format(_h)

plt.subplot(111)
plt.grid(''); plt.xticks([]); plt.yticks([])

img = images[15]['image']

plt.imshow(img, cmap = 'bone_r', vmin = 0., vmax = 1.)


plt.xlim(0, edge_len)
plt.ylim(0, edge_len)

#plt.tight_layout()
plt.subplots_adjust(left = .0, right = 1., top = 1., bottom = .0)

plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
''' save all generated images'''

# define output path
out_path = '../data/subj-refr-foc/'

# store all images to file
optics.store_images(images, out_path)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
import scipy.ndimage

In [31]:

# iterate each image in batch
for image in images[:]:

    # get current image data
    _img = image['image']
    
    # get image delta value
    d = image['delta']
    
    # generate zoom
    rotate = np.arange(-90., 90., 5.)
    
    for r in rotate:
        
        __img = ndimage.rotate(_img, r, reshape = False)

        # initialise figure and axes, clean format
        _w = 4; _h = 4
        fig = plt.figure(figsize = (_w, _h))
        fig.canvas.layout.width = '{}in'.format(_w)
        fig.canvas.layout.height= '{}in'.format(_h)

        plt.subplot(111)
        plt.grid(''); plt.xticks([]); plt.yticks([])

        plt.imshow(__img, cmap = 'bone_r', vmin = 0., vmax = 1.)


        #plt.xlim(0, edge_len)
        #plt.ylim(0, edge_len)
        plt.xlim(0, 301)
        plt.ylim(0, 301)
        
        #plt.tight_layout()
        plt.subplots_adjust(left = .0, right = 1., top = 1., bottom = .0)

        plt.savefig('../data/subj-refr-high/subj-refr_pow-{:.3f}_axs-{:.1f}.png'.format(d, r), dpi = 250)

        plt.close()

        

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:


image = images[0]

# get current image data
_img = image['image']

# get image delta value
d = image['delta']

# generate zoom
rotate = np.arange(-90., 90., 10.)
    
r = rotate[3]
        
__img = ndimage.rotate(_img, r)

# initialise figure and axes, clean format
_w = 4; _h = 4
fig = plt.figure(figsize = (_w, _h))
fig.canvas.layout.width = '{}in'.format(_w)
fig.canvas.layout.height= '{}in'.format(_h)

plt.subplot(111)
plt.grid(''); plt.xticks([]); plt.yticks([])

plt.imshow(__img, cmap = 'bone_r', vmin = 0., vmax = 1.)


plt.xlim(0, 151)
plt.ylim(0, 151)

#plt.tight_layout()
plt.subplots_adjust(left = .0, right = 1., top = 1., bottom = .0)

#plt.savefig('../data/subj-refr/subj-refr_pow-{:.3f}_axs-{:.1f}.png'.format(d, r), dpi = 200)

plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
# initialise figure and axes, clean format
_w = 7; _h = 4
fig = plt.figure(figsize = (_w, _h))
fig.canvas.layout.width = '{}in'.format(_w)
fig.canvas.layout.height= '{}in'.format(_h)

plt.subplot(121)
plt.grid(''); plt.xticks([]); plt.yticks([])

_img = [ images[i]['image'] for i in range(len(images)) if images[i]['delta'] > 0. ][0]

plt.imshow(_img, cmap = 'bone_r', vmin = 0., vmax = 1.)


plt.subplot(122)
plt.grid(''); plt.xticks([]); plt.yticks([])

_img = images[0]['image']

plt.imshow(_img, cmap = 'bone_r', vmin = 0., vmax = 1.)


#plt.tight_layout()
plt.subplots_adjust(left = .0, right = 1., top = 1., bottom = .0)

#plt.savefig('./images/output-0s95.png', dpi = 200)

plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
## plot ray paths

# initialise 3d figure
_w = 7; _h = 6
fig = plt.figure(figsize = (_w, _h))
fig.canvas.layout.width = '{}in'.format(_w)
fig.canvas.layout.height= '{}in'.format(_h)
ax = fig.add_subplot(111, projection='3d')

#ax.set_xlim(0., 250.)
#ax.set_ylim(-15., 15.)
#ax.set_zlim(-15., 15.)


# plot all optics
if True:
    # iterate over each optic in chain
    for optic in opts[:2]:
        
        # get optic parameters
        C = optic['centre']
        r = optic['radius']
        e = optic['scale']
        n2 = optic['opt_den']
        rev = optic['rev']
        theta = optic['theta']
        
        print(theta)

        # get optic points in 3d for plotting
        x, y, z = optics.plot_3d_ellipsoid(C, r, e, rev, theta)
        # plot ellipsoid
        ax.plot_wireframe(x, y, z,  rstride = 2, cstride = 2, color = 'k', alpha = 0.5)


# iterate over each ray path
for i in range(len(paths))[::200]:
    
    # check for any refraction of ray with optics
    #if len(paths[i]) > 1:
    # only ray that hit retina
    if len(paths[i]) == 7:
        path = paths[i]
        
        # iterate ray path through optics
        for j in range(len(path)-1)[1:]:
            
            # plot path segment
            ax.plot([path[j][0][0], path[j+1][0][0]],
                    [path[j][0][1], path[j+1][0][1]],
                    [path[j][0][2], path[j+1][0][2]],
                    color = 'r', alpha = 0.7)

# format and display figure
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.0
0.0


In [34]:
# initialise 3d figure
_w = 7; _h = 6
fig = plt.figure(figsize = (_w, _h))
fig.canvas.layout.width = '{}in'.format(_w)
fig.canvas.layout.height= '{}in'.format(_h)
ax = fig.add_subplot(111, projection='3d')

#ax.set_xlim(0., 50.)
#ax.set_ylim(-15., 15.)
#ax.set_zlim(-15., 15.)


# plot all optics
if True:
    # iterate over each optic in chain
    for optic in rev_opts[-2:-1]:
        
        # get optic parameters
        C = optic['centre']
        r = optic['radius']
        e = optic['scale']
        n2 = optic['opt_den']
        rev = optic['rev']
        theta = optic['theta']

        # get optic points in 3d for plotting
        x, y, z = optics.plot_3d_ellipsoid(C, r, e, rev, theta)
        # plot ellipsoid
        ax.plot_wireframe(x, y, z,  rstride = 2, cstride = 2, color = 'k', alpha = 0.5)


# iterate over each ray path
for i in range(len(rev_paths))[::100]:
    
    # check for any refraction of ray with optics
    if len(paths[i]) > 1:
    # only ray that hit retina
    #if len(paths[i]) == 7:
        path = rev_paths[i]
        
        # iterate ray path through optics
        for j in range(len(path)-1)[:-1]:
            
            # plot path segment
            ax.plot([path[j][0][0], path[j+1][0][0]],
                    [path[j][0][1], path[j+1][0][1]],
                    [path[j][0][2], path[j+1][0][2]],
                    color = 'r', alpha = 0.7)

# format and display figure
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …